In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:9 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
# Deliverable 2 - Extract the dataset used in Deliverable 1

from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [4]:
# Recreate the vine_table, and perform your analysis using the steps above.
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RTIS3L2M1F5SM|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|R3BH071QLH8QMC|          1|            0|          1|   N|                Y|
|R127K9NTSXA2YH|          3|            0|          0|   N|                Y|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|                Y|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|                Y|
|R2F0POU5K6F73F|          5|            0|          0|   N|                Y|
|R3VNR804HYSMR6|          5|            0|          0|   N|                Y|
| R3GZTM72WA2QH|          5|            0|          0|   N|                Y|
| RNQOY62705W1K|          4|            0|          0|   N|     

In [5]:
# 1. Filter the data and create a new DataFrame or table to retrieve all the rows
# where the total_votes count is equal to or greater than 20 to pick reviews that
# are more likely to be helpful and to avoid having division by zero errors later on.
vine_df = vine_df.filter("total_votes>20")
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R3EZ0EPYLDA34S|          1|           14|         31|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R3GSK9MM8DNOYI|          1|            4|         32|   N|     

In [6]:
# 2. Filter the new DataFrame or table created in Step 1 and create a new DataFrame
# or table to retrieve all the rows where the number of helpful_votes divided by
# total_votes is equal to or greater than 50%.
vine_df = vine_df.withColumn('helpful_50', (vine_df['helpful_votes']/vine_df['total_votes'])*100).filter('helpful_50>50')
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|       helpful_50|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|61.76470588235294|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|             60.0|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|             84.0|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|91.66666666666666|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|78.46153846153847|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|86.11111111111111|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|55.88235294117647|
|R36O341WWXXKNP|          5|           2

In [7]:
# 3. Filter the DataFrame or table created in Step 2, and create a new DataFrame
# or table that retrieves all the rows where a review was written as part of the
# Vine program (paid), vine == 'Y'.
vine_paid_df = vine_df.filter("vine=='Y'")
vine_paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|       helpful_50|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|R3KKUSGFZWSUIY|          5|           56|         63|   Y|                N|88.88888888888889|
|R10FO5UKKVZBK2|          3|           23|         23|   Y|                N|            100.0|
| RM4KSGEOR7MU1|          5|           19|         24|   Y|                N|79.16666666666666|
| RG7VRMYLEXD23|          4|           22|         26|   Y|                N|84.61538461538461|
|R11O4YSCPSNL6L|          3|           20|         26|   Y|                N|76.92307692307693|
|R286MFBAJ8NPD6|          5|           46|         51|   Y|                N|90.19607843137256|
|R1JRR530H4COA2|          5|           22|         28|   Y|                N|78.57142857142857|
| RQ5WD90PUNBU9|          5|           2

In [8]:
# 4. Repeat Step 3, but this time retrieve all the rows where the review was not
# part of the Vine program (unpaid), vine == 'N'.
vine_unpaid_df = vine_df.filter("vine=='N'")
vine_unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|       helpful_50|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|61.76470588235294|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|             60.0|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|             84.0|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|91.66666666666666|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|78.46153846153847|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|86.11111111111111|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|55.88235294117647|
|R36O341WWXXKNP|          5|           2

In [9]:
# 5. Determine the total number of reviews, the number of 5-star reviews, and
# the percentage of 5-star reviews for the two types of review (paid vs unpaid).

In [10]:
from pyspark.sql.functions import count
count_vine_df = vine_df.groupby('vine').agg(count('vine')).withColumnRenamed('count(vine)', "total")
total_paid = count_vine_df.filter(count_vine_df.vine.contains('Y')).withColumnRenamed('vine', 'paid')
total_unpaid = count_vine_df.filter(count_vine_df.vine.contains('N')).withColumnRenamed('vine', 'paid')
count_vine_df.show()
total_paid.show()
total_unpaid.show()

+----+-----+
|vine|total|
+----+-----+
|   Y|   90|
|   N|37385|
+----+-----+

+----+-----+
|paid|total|
+----+-----+
|   Y|   90|
+----+-----+

+----+-----+
|paid|total|
+----+-----+
|   N|37385|
+----+-----+



In [11]:
paid_count_df = vine_paid_df.groupby('star_rating').count()
paid_count_df = paid_count_df.filter('star_rating==5').withColumnRenamed('count', 'five_star').select('five_star')
paid_count_df = total_paid.join(paid_count_df).select('paid', 'five_star', 'total')

paid_count_df.show()

+----+---------+-----+
|paid|five_star|total|
+----+---------+-----+
|   Y|       44|   90|
+----+---------+-----+



In [12]:
unpaid_count_df = vine_unpaid_df.groupby('star_rating').count()
unpaid_count_df = unpaid_count_df.filter('star_rating==5').withColumnRenamed('count', 'five_star').select('five_star')
unpaid_count_df = total_unpaid.join(unpaid_count_df).select('paid', 'five_star', 'total')

unpaid_count_df.show()

+----+---------+-----+
|paid|five_star|total|
+----+---------+-----+
|   N|    14626|37385|
+----+---------+-----+



In [13]:
joined_df = paid_count_df.union(unpaid_count_df)
joined_df = joined_df.withColumn('percentage', (joined_df['five_star']/joined_df['total'])*100)

joined_df.show()

+----+---------+-----+------------------+
|paid|five_star|total|        percentage|
+----+---------+-----+------------------+
|   Y|       44|   90|48.888888888888886|
|   N|    14626|37385|39.122642771164905|
+----+---------+-----+------------------+



# **Summary - Part 2. Provide one additional analysis that you could do with the dataset to support your statement.**

In [14]:
summary_df = vine_unpaid_df

verified_count_df = summary_df.groupby('verified_purchase').agg(count('verified_purchase')).withColumnRenamed('count(verified_purchase)', "total")

verified_df = verified_count_df.filter(verified_count_df.verified_purchase.contains('Y'))
verified_df.show()


+-----------------+-----+
|verified_purchase|total|
+-----------------+-----+
|                Y| 9355|
+-----------------+-----+



In [15]:
verified_unpaid_df = vine_unpaid_df.filter("verified_purchase=='Y'")
verified_unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|       helpful_50|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|             60.0|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|             84.0|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|78.46153846153847|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|55.88235294117647|
|R1VR5GLGY1GE7N|          1|           49|         51|   N|                Y|96.07843137254902|
|R16621F16PQN5A|          3|           24|         31|   N|                Y|77.41935483870968|
| RNEMGSUKJ4535|          2|           21|         25|   N|                Y|             84.0|
|R3R2UKBFKP1IQ4|          1|           1

In [16]:
ver_unpaid_count_df = verified_unpaid_df.groupby('star_rating').count()
ver_unpaid_count_df = ver_unpaid_count_df.filter('star_rating==5').withColumnRenamed('count', 'five_star').select('five_star')
ver_unpaid_count_df.show()

ver_count_vine_df = verified_unpaid_df.groupby('vine').agg(count('vine')).withColumnRenamed('count(vine)', "total")
ver_total_unpaid = ver_count_vine_df.withColumnRenamed('vine', 'paid')
ver_total_unpaid.show()

ver_join_unpaid_count_df = ver_total_unpaid.join(ver_unpaid_count_df).select('paid', 'five_star', 'total')
ver_join_unpaid_count_df.show()

ver_joined_df = paid_count_df.union(ver_join_unpaid_count_df)
ver_joined_df = ver_joined_df.withColumn('percentage', (ver_joined_df['five_star']/ver_joined_df['total'])*100)

ver_joined_df.show()

+---------+
|five_star|
+---------+
|     3594|
+---------+

+----+-----+
|paid|total|
+----+-----+
|   N| 9355|
+----+-----+

+----+---------+-----+
|paid|five_star|total|
+----+---------+-----+
|   N|     3594| 9355|
+----+---------+-----+

+----+---------+-----+------------------+
|paid|five_star|total|        percentage|
+----+---------+-----+------------------+
|   Y|       44|   90|48.888888888888886|
|   N|     3594| 9355| 38.41795831106361|
+----+---------+-----+------------------+

